In [1]:
# Install required packages
!pip install tensorflow keras numpy pandas flask flask-cors Pillow


# Import libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from PIL import Image
from flask import Flask, request, jsonify

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rkuo2000/uecfood100")

print("Path to dataset files:", path)

100%|██████████| 946M/946M [00:25<00:00, 38.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rkuo2000/uecfood100/versions/1


In [3]:
import shutil

source_dir = '/root/.cache/kagglehub/datasets/rkuo2000/uecfood100/versions/1'
dest_dir = '/content/food-101'

shutil.move(source_dir, dest_dir)


'/content/food-101'

In [4]:
import os
import shutil
import random

# Path to the folder where your downloaded dataset currently resides
# (it should contain one folder per class)
data_dir = '/content/food-101'

# Create the destination train and test directories
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# List all class directories (skip the newly created train/test folders)
class_names = [d for d in os.listdir(data_dir)
               if os.path.isdir(os.path.join(data_dir, d)) and d not in ['train', 'test']]

# For each class folder, split images 80% for train and 20% for test
for class_name in class_names:
    class_path = os.path.join(data_dir, class_name)

    # Create subdirectories for this class in both train and test directories
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Get list of image files (adjust the filter if needed)
    images = [f for f in os.listdir(class_path)
              if os.path.isfile(os.path.join(class_path, f))]

    # Shuffle and split images into train and test sets
    random.shuffle(images)
    split_idx = int(0.8 * len(images))
    train_images = images[:split_idx]
    test_images = images[split_idx:]

    # Copy images to the corresponding train folder
    for img in train_images:
        src = os.path.join(class_path, img)
        dst = os.path.join(train_class_dir, img)
        shutil.copy(src, dst)

    # Copy images to the corresponding test folder
    for img in test_images:
        src = os.path.join(class_path, img)
        dst = os.path.join(test_class_dir, img)
        shutil.copy(src, dst)

print("Dataset successfully split into train and test directories.")


Dataset successfully split into train and test directories.


In [5]:
import os
import shutil
import random

# Define paths
# This is the folder that contains the 100 food category directories
source_dir = '/content/food-101/UECFOOD100'
# These folders were already created by your previous code
train_dir = '/content/food-101/train'
test_dir = '/content/food-101/test'

# List only the subdirectories that represent the food classes.
# Assuming class directories are named with digits (e.g., '1', '2', ..., '100')
categories = [d for d in os.listdir(source_dir)
              if os.path.isdir(os.path.join(source_dir, d)) and d.isdigit()]

for category in categories:
    cat_source = os.path.join(source_dir, category)

    # Create corresponding subdirectories in train and test folders
    cat_train = os.path.join(train_dir, category)
    cat_test = os.path.join(test_dir, category)
    os.makedirs(cat_train, exist_ok=True)
    os.makedirs(cat_test, exist_ok=True)

    # List only image files ending with '.jpg'
    image_files = [f for f in os.listdir(cat_source)
                   if os.path.isfile(os.path.join(cat_source, f)) and f.lower().endswith('.jpg')]

    # Shuffle and split files: 80% for training, 20% for testing
    random.shuffle(image_files)
    split_idx = int(0.8 * len(image_files))
    train_files = image_files[:split_idx]
    test_files = image_files[split_idx:]

    # Copy training images
    for img in train_files:
        src_path = os.path.join(cat_source, img)
        dst_path = os.path.join(cat_train, img)
        shutil.copy(src_path, dst_path)

    # Copy testing images
    for img in test_files:
        src_path = os.path.join(cat_source, img)
        dst_path = os.path.join(cat_test, img)
        shutil.copy(src_path, dst_path)

print("Files copied into train and test directories for each category.")


Files copied into train and test directories for each category.


In [6]:
# Load Food-101 dataset
train_dir = '/content/food-101/train'
test_dir = '/content/food-101/test'

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training')

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

Found 9199 images belonging to 101 classes.
Found 2249 images belonging to 101 classes.


In [7]:
# Create EfficientNet model
base_model = tf.keras.applications.EfficientNetB3(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet')

model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(101, activation='softmax')
])

model.compile(optimizer=keras.optimizers.Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator))

# Save model
model.save('food_recognition.h5')

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
288/288 ━━━━━━━━━━━━━━━━━━━━ 376s 857ms/step - accuracy: 0.0609 - loss: 4.4074 - val_accuracy: 0.0325 - val_loss: 4.5248
Epoch 2/15
288/288 ━━━━━━━━━━━━━━━━━━━━ 183s 600ms/step - accuracy: 0.3369 - loss: 2.9169 - val_accuracy: 0.0751 - val_loss: 4.2310
Epoch 3/15
288/288 ━━━━━━━━━━━━━━━━━━━━ 172s 595ms/step - accuracy: 0.5454 - loss: 1.8070 - val_accuracy: 0.1565 - val_loss: 4.0236
Epoch 4/15
288/288 ━━━━━━━━━━━━━━━━━━━━ 173s 600ms/step - accuracy: 0.6563 - loss: 1.2805 - val_accuracy: 0.1423 - val_loss: 4.0818
Epoch 5/15
288/288 ━━━━━━━━━━━━━━━━━━━━ 174s 605ms/step - accuracy: 0.7089 - loss: 1.0354 - val_accuracy: 0.2201 - val_loss: 3.7783
Epoch 6/15
288/288 ━━━━━━━━━━━━━━━━━━━━ 200s 598ms/step - accuracy: 0.7489 - loss: 0.8715 - val_accuracy: 0.0258 - val_loss: 20.7016
Epoch 7/15
288/288 ━━━━━━━━━━━━━━━━━━━━ 201s 596ms/step - accuracy: 0.7837 - loss: 0.7332 - val_accuracy: 0.4206 - val_loss: 2.6366
Epoch 8/15
288/288 ━━━━━━━━━━━━━━━━━━━━ 172s 599ms/step - accuracy: 0.8086 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("utsavdey1410/food-nutrition-dataset")

print("Path to dataset files:", path)

100%|██████████| 694k/694k [00:00<00:00, 91.7MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/utsavdey1410/food-nutrition-dataset/versions/1


In [ ]:
import shutil

source_dir = '/root/.cache/kagglehub/datasets/utsavdey1410/food-nutrition-dataset/versions/1'
dest_dir = '/content/FINAL FOOD DATASET'

shutil.move(source_dir, dest_dir)


'/content/FINAL FOOD DATASET'

In [ ]:
df1 = pd.read_csv(r"/content/FINAL FOOD DATASET/FINAL FOOD DATASET/FOOD-DATA-GROUP1.csv")
df2 = pd.read_csv(r"/content/FINAL FOOD DATASET/FINAL FOOD DATASET/FOOD-DATA-GROUP2.csv")
df3 = pd.read_csv(r"/content/FINAL FOOD DATASET/FINAL FOOD DATASET/FOOD-DATA-GROUP3.csv")
df4 = pd.read_csv(r"/content/FINAL FOOD DATASET/FINAL FOOD DATASET/FOOD-DATA-GROUP3.csv")
df5 = pd.read_csv(r"/content/FINAL FOOD DATASET/FINAL FOOD DATASET/FOOD-DATA-GROUP5.csv")

In [ ]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [ ]:
df.shape

(2012, 37)

In [ ]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'food', 'Caloric Value', 'Fat',
       'Saturated Fats', 'Monounsaturated Fats', 'Polyunsaturated Fats',
       'Carbohydrates', 'Sugars', 'Protein', 'Dietary Fiber', 'Cholesterol',
       'Sodium', 'Water', 'Vitamin A', 'Vitamin B1', 'Vitamin B11',
       'Vitamin B12', 'Vitamin B2', 'Vitamin B3', 'Vitamin B5', 'Vitamin B6',
       'Vitamin C', 'Vitamin D', 'Vitamin E', 'Vitamin K', 'Calcium', 'Copper',
       'Iron', 'Magnesium', 'Manganese', 'Phosphorus', 'Potassium', 'Selenium',
       'Zinc', 'Nutrition Density'],
      dtype='object')

In [ ]:
# Step 4: Complete Nutrition Integration with Full Class Map

# Create complete class mapping (ID to Food Name)
class_map = [
    {"id": 1, "name": "rice"},
    {"id": 2, "name": "eels on rice"},
    {"id": 3, "name": "pilaf"},
    {"id": 4, "name": "chicken-'n'-egg on rice"},
    {"id": 5, "name": "pork cutlet on rice"},
    {"id": 6, "name": "beef curry"},
    {"id": 7, "name": "sushi"},
    {"id": 8, "name": "chicken rice"},
    {"id": 9, "name": "fried rice"},
    {"id": 10, "name": "tempura bowl"},
    {"id": 11, "name": "bibimbap"},
    {"id": 12, "name": "toast"},
    {"id": 13, "name": "croissant"},
    {"id": 14, "name": "roll bread"},
    {"id": 15, "name": "raisin bread"},
    {"id": 16, "name": "chip butty"},
    {"id": 17, "name": "hamburger"},
    {"id": 18, "name": "pizza"},
    {"id": 19, "name": "sandwiches"},
    {"id": 20, "name": "udon noodle"},
    {"id": 21, "name": "tempura udon"},
    {"id": 22, "name": "soba noodle"},
    {"id": 23, "name": "ramen noodle"},
    {"id": 24, "name": "beef noodle"},
    {"id": 25, "name": "tensin noodle"},
    {"id": 26, "name": "fried noodle"},
    {"id": 27, "name": "spaghetti"},
    {"id": 28, "name": "Japanese-style pancake"},
    {"id": 29, "name": "takoyaki"},
    {"id": 30, "name": "gratin"},
    {"id": 31, "name": "sauteed vegetables"},
    {"id": 32, "name": "croquette"},
    {"id": 33, "name": "grilled eggplant"},
    {"id": 34, "name": "sauteed spinach"},
    {"id": 35, "name": "vegetable tempura"},
    {"id": 36, "name": "miso soup"},
    {"id": 37, "name": "potage"},
    {"id": 38, "name": "sausage"},
    {"id": 39, "name": "oden"},
    {"id": 40, "name": "omelet"},
    {"id": 41, "name": "ganmodoki"},
    {"id": 42, "name": "jiaozi"},
    {"id": 43, "name": "stew"},
    {"id": 44, "name": "teriyaki grilled fish"},
    {"id": 45, "name": "fried fish"},
    {"id": 46, "name": "grilled salmon"},
    {"id": 47, "name": "salmon meuniere"},
    {"id": 48, "name": "sashimi"},
    {"id": 49, "name": "grilled pacific saury"},
    {"id": 50, "name": "sukiyaki"},
    {"id": 51, "name": "sweet and sour pork"},
    {"id": 52, "name": "lightly roasted fish"},
    {"id": 53, "name": "steamed egg hotchpotch"},
    {"id": 54, "name": "tempura"},
    {"id": 55, "name": "fried chicken"},
    {"id": 56, "name": "sirloin cutlet"},
    {"id": 57, "name": "nanbanzuke"},
    {"id": 58, "name": "boiled fish"},
    {"id": 59, "name": "seasoned beef with potatoes"},
    {"id": 60, "name": "hambarg steak"},
    {"id": 61, "name": "beef steak"},
    {"id": 62, "name": "dried fish"},
    {"id": 63, "name": "ginger pork saute"},
    {"id": 64, "name": "spicy chili-flavored tofu"},
    {"id": 65, "name": "yakitori"},
    {"id": 66, "name": "cabbage roll"},
    {"id": 67, "name": "rolled omelet"},
    {"id": 68, "name": "egg sunny-side up"},
    {"id": 69, "name": "fermented soybeans"},
    {"id": 70, "name": "cold tofu"},
    {"id": 71, "name": "egg roll"},
    {"id": 72, "name": "chilled noodle"},
    {"id": 73, "name": "stir-fried beef and peppers"},
    {"id": 74, "name": "simmered pork"},
    {"id": 75, "name": "boiled chicken and vegetables"},
    {"id": 76, "name": "sashimi bowl"},
    {"id": 77, "name": "sushi bowl"},
    {"id": 78, "name": "fish-shaped pancake with bean jam"},
    {"id": 79, "name": "shrimp with chill source"},
    {"id": 80, "name": "roast chicken"},
    {"id": 81, "name": "steamed meat dumpling"},
    {"id": 82, "name": "omelet with fried rice"},
    {"id": 83, "name": "cutlet curry"},
    {"id": 84, "name": "spaghetti meat sauce"},
    {"id": 85, "name": "fried shrimp"},
    {"id": 86, "name": "potato salad"},
    {"id": 87, "name": "green salad"},
    {"id": 88, "name": "macaroni salad"},
    {"id": 89, "name": "Japanese tofu and vegetable chowder"},
    {"id": 90, "name": "pork miso soup"},
    {"id": 91, "name": "chinese soup"},
    {"id": 92, "name": "beef bowl"},
    {"id": 93, "name": "kinpira-style sauteed burdock"},
    {"id": 94, "name": "rice ball"},
    {"id": 95, "name": "pizza toast"},
    {"id": 96, "name": "dipping noodles"},
    {"id": 97, "name": "hot dog"},
    {"id": 98, "name": "french fries"},
    {"id": 99, "name": "mixed rice"},
    {"id": 100, "name": "goya chanpuru"}
]

class_df = pd.DataFrame(class_map)

# Load and clean nutrition data
df1 = pd.read_csv("/content/FINAL FOOD DATASET/FINAL FOOD DATASET/FOOD-DATA-GROUP1.csv")
df2 = pd.read_csv("/content/FINAL FOOD DATASET/FINAL FOOD DATASET/FOOD-DATA-GROUP2.csv")
df3 = pd.read_csv("/content/FINAL FOOD DATASET/FINAL FOOD DATASET/FOOD-DATA-GROUP3.csv")
df4 = pd.read_csv("/content/FINAL FOOD DATASET/FINAL FOOD DATASET/FOOD-DATA-GROUP4.csv")  # Fixed
df5 = pd.read_csv("/content/FINAL FOOD DATASET/FINAL FOOD DATASET/FOOD-DATA-GROUP5.csv")

nutrition_df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

# Clean nutrition data
nutrition_df = nutrition_df.rename(columns={'food': 'Food'})
nutrition_df['Food'] = nutrition_df['Food'].str.lower().str.strip()
nutrition_df = nutrition_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'Nutrition Density'], errors='ignore')

# Create improved nutrition mapping with fuzzy matching
!pip install fuzzywuzzy python-Levenshtein
from fuzzywuzzy import process

nutrition_mapping = {}
food_choices = nutrition_df['Food'].unique().tolist()

for _, row in class_df.iterrows():
    target_name = row['name'].lower()

    # Fuzzy match with score threshold
    match, score = process.extractOne(target_name, food_choices, scorer=process.fuzz.token_sort_ratio)

    if score > 75:  # Adjust threshold as needed
        nutrition_data = nutrition_df[nutrition_df['Food'] == match].iloc[0].to_dict()
        nutrition_mapping[row['id']] = nutrition_data
    else:
        print(f"No match found for {row['name']} (best match: {match}@{score})")
        nutrition_mapping[row['id']] = None

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.5 MB/s eta 0:00:00
No match found for rice (best match: corn rice@62)
No match found for eels on rice (best match: corn rice@67)
No match found for pilaf (best match: sapodilla@57)
No match found for chicken-'n'-egg on rice (best match: chicken egg roll@70)
No match found for pork cutlet on rice (best match: pork loin raw@62)
No match found for beef curry (best match: beef jerky@70)
No match found for sushi (best match: succotash@57)
No match found for tempura bowl (best match: black tea@57)
No match found for bibimbap (best match: biscuit mcdonalds@40)
No match found for toast (best match: oats@67)
No match found for croissant (best match: apple croissant@75)
No match found for roll bread (best match: rye bread@74)
No match found for chip butty (best match: cashew butter@61)
No match found for pizza (best match: pizza sauce@62)
No match found for sa

In [8]:
 MANUAL_NUTRITION_DB = {
 1: {'name': 'rice', 'calories': 130, 'protein': 2.7, 'carbs': 28, 'fat': 0.3},
    2: {'name': 'eels on rice', 'calories': 450, 'protein': 20, 'carbs': 60, 'fat': 12},
    3: {'name': 'pilaf', 'calories': 350, 'protein': 8, 'carbs': 45, 'fat': 15},
    4: {'name': 'chicken-\'n\'-egg on rice', 'calories': 600, 'protein': 25, 'carbs': 75, 'fat': 20},
    5: {'name': 'pork cutlet on rice', 'calories': 700, 'protein': 30, 'carbs': 80, 'fat': 25},
    6: {'name': 'beef curry', 'calories': 650, 'protein': 22, 'carbs': 70, 'fat': 28},
    7: {'name': 'sushi', 'calories': 200, 'protein': 12, 'carbs': 40, 'fat': 4},  # 4 pieces
    8: {'name': 'chicken rice', 'calories': 400, 'protein': 20, 'carbs': 45, 'fat': 15},
    9: {'name': 'fried rice', 'calories': 500, 'protein': 15, 'carbs': 60, 'fat': 20},
    10: {'name': 'tempura bowl', 'calories': 750, 'protein': 18, 'carbs': 85, 'fat': 35},
    11: {'name': 'bibimbap', 'calories': 550, 'protein': 20, 'carbs': 65, 'fat': 20},

    # Breads (12-16)
    12: {'name': 'toast', 'calories': 75, 'protein': 3, 'carbs': 12, 'fat': 1},
    13: {'name': 'croissant', 'calories': 230, 'protein': 5, 'carbs': 25, 'fat': 12},
    14: {'name': 'roll bread', 'calories': 150, 'protein': 4, 'carbs': 28, 'fat': 2},
    15: {'name': 'raisin bread', 'calories': 180, 'protein': 4, 'carbs': 33, 'fat': 3},
    16: {'name': 'chip butty', 'calories': 400, 'protein': 7, 'carbs': 45, 'fat': 22},

    # Western Dishes (17-19)
    17: {'name': 'hamburger', 'calories': 295, 'protein': 17, 'carbs': 24, 'fat': 14},
    18: {'name': 'pizza', 'calories': 1140, 'protein': 48, 'carbs': 132, 'fat': 40},  # whole pizza
    19: {'name': 'sandwiches', 'calories': 300, 'protein': 15, 'carbs': 30, 'fat': 12},

    # Noodles (20-27)
    20: {'name': 'udon noodle', 'calories': 350, 'protein': 10, 'carbs': 60, 'fat': 5},
    21: {'name': 'tempura udon', 'calories': 600, 'protein': 15, 'carbs': 75, 'fat': 25},
    22: {'name': 'soba noodle', 'calories': 300, 'protein': 12, 'carbs': 55, 'fat': 4},
    23: {'name': 'ramen noodle', 'calories': 400, 'protein': 15, 'carbs': 65, 'fat': 12},
    24: {'name': 'beef noodle', 'calories': 450, 'protein': 20, 'carbs': 50, 'fat': 18},
    25: {'name': 'tensin noodle', 'calories': 380, 'protein': 18, 'carbs': 55, 'fat': 10},
    26: {'name': 'fried noodle', 'calories': 450, 'protein': 12, 'carbs': 60, 'fat': 18},
    27: {'name': 'spaghetti', 'calories': 220, 'protein': 8, 'carbs': 43, 'fat': 1},

    # Japanese Specialties (28-50)
    28: {'name': 'Japanese-style pancake', 'calories': 350, 'protein': 15, 'carbs': 30, 'fat': 20},
    29: {'name': 'takoyaki', 'calories': 320, 'protein': 20, 'carbs': 40, 'fat': 12},  # 4 pieces
    30: {'name': 'gratin', 'calories': 400, 'protein': 18, 'carbs': 35, 'fat': 22},
    31: {'name': 'sauteed vegetables', 'calories': 120, 'protein': 3, 'carbs': 15, 'fat': 7},
    32: {'name': 'croquette', 'calories': 150, 'protein': 5, 'carbs': 15, 'fat': 8},  # per piece
    33: {'name': 'grilled eggplant', 'calories': 50, 'protein': 1, 'carbs': 12, 'fat': 0.2},
    34: {'name': 'sauteed spinach', 'calories': 40, 'protein': 3, 'carbs': 6, 'fat': 1},
    35: {'name': 'vegetable tempura', 'calories': 320, 'protein': 12, 'carbs': 32, 'fat': 16},  # 4 pieces
    36: {'name': 'miso soup', 'calories': 35, 'protein': 2, 'carbs': 5, 'fat': 1},
    37: {'name': 'potage', 'calories': 150, 'protein': 4, 'carbs': 20, 'fat': 6},
    38: {'name': 'sausage', 'calories': 150, 'protein': 7, 'carbs': 2, 'fat': 13},
    39: {'name': 'oden', 'calories': 150, 'protein': 8, 'carbs': 12, 'fat': 7},
    40: {'name': 'omelet', 'calories': 154, 'protein': 11, 'carbs': 2, 'fat': 12},
     # Continued from previous
    41: {'name': 'ganmodoki', 'calories': 180, 'protein': 8, 'carbs': 15, 'fat': 12},
    42: {'name': 'jiaozi', 'calories': 50, 'protein': 3, 'carbs': 6, 'fat': 2},  # per dumpling
    43: {'name': 'stew', 'calories': 250, 'protein': 15, 'carbs': 20, 'fat': 12},
    44: {'name': 'teriyaki grilled fish', 'calories': 280, 'protein': 25, 'carbs': 15, 'fat': 14},
    45: {'name': 'fried fish', 'calories': 350, 'protein': 20, 'carbs': 25, 'fat': 20},
    46: {'name': 'grilled salmon', 'calories': 300, 'protein': 27, 'carbs': 0, 'fat': 18},
    47: {'name': 'salmon meuniere', 'calories': 320, 'protein': 28, 'carbs': 10, 'fat': 20},
    48: {'name': 'sashimi', 'calories': 120, 'protein': 20, 'carbs': 0, 'fat': 5},  # per 100g
    49: {'name': 'grilled pacific saury', 'calories': 250, 'protein': 22, 'carbs': 0, 'fat': 17},
    50: {'name': 'sukiyaki', 'calories': 600, 'protein': 35, 'carbs': 25, 'fat': 35},

    51: {'name': 'sweet and sour pork', 'calories': 380, 'protein': 18, 'carbs': 35, 'fat': 20},
    52: {'name': 'lightly roasted fish', 'calories': 200, 'protein': 25, 'carbs': 0, 'fat': 10},
    53: {'name': 'steamed egg hotchpotch', 'calories': 150, 'protein': 12, 'carbs': 8, 'fat': 9},
    54: {'name': 'tempura', 'calories': 80, 'protein': 3, 'carbs': 8, 'fat': 4},  # per piece
    55: {'name': 'fried chicken', 'calories': 320, 'protein': 20, 'carbs': 15, 'fat': 22},
    56: {'name': 'sirloin cutlet', 'calories': 450, 'protein': 30, 'carbs': 20, 'fat': 28},
    57: {'name': 'nanbanzuke', 'calories': 280, 'protein': 18, 'carbs': 15, 'fat': 16},
    58: {'name': 'boiled fish', 'calories': 180, 'protein': 25, 'carbs': 0, 'fat': 8},
    59: {'name': 'seasoned beef with potatoes', 'calories': 400, 'protein': 22, 'carbs': 35, 'fat': 20},
    60: {'name': 'hambarg steak', 'calories': 350, 'protein': 25, 'carbs': 15, 'fat': 24},

    61: {'name': 'beef steak', 'calories': 400, 'protein': 35, 'carbs': 0, 'fat': 28},
    62: {'name': 'dried fish', 'calories': 150, 'protein': 30, 'carbs': 0, 'fat': 3},
    63: {'name': 'ginger pork saute', 'calories': 380, 'protein': 28, 'carbs': 15, 'fat': 25},
    64: {'name': 'spicy chili-flavored tofu', 'calories': 180, 'protein': 12, 'carbs': 10, 'fat': 10},
    65: {'name': 'yakitori', 'calories': 120, 'protein': 10, 'carbs': 3, 'fat': 8},  # per skewer
    66: {'name': 'cabbage roll', 'calories': 200, 'protein': 12, 'carbs': 15, 'fat': 10},
    67: {'name': 'rolled omelet', 'calories': 150, 'protein': 10, 'carbs': 5, 'fat': 10},
    68: {'name': 'egg sunny-side up', 'calories': 90, 'protein': 6, 'carbs': 0, 'fat': 7},
    69: {'name': 'fermented soybeans', 'calories': 200, 'protein': 15, 'carbs': 20, 'fat': 8},
    70: {'name': 'cold tofu', 'calories': 80, 'protein': 8, 'carbs': 2, 'fat': 5},

    71: {'name': 'egg roll', 'calories': 100, 'protein': 7, 'carbs': 5, 'fat': 6},  # per piece
    72: {'name': 'chilled noodle', 'calories': 300, 'protein': 10, 'carbs': 50, 'fat': 8},
    73: {'name': 'stir-fried beef and peppers', 'calories': 280, 'protein': 22, 'carbs': 15, 'fat': 16},
    74: {'name': 'simmered pork', 'calories': 350, 'protein': 25, 'carbs': 10, 'fat': 25},
    75: {'name': 'boiled chicken and vegetables', 'calories': 250, 'protein': 20, 'carbs': 20, 'fat': 10},
    76: {'name': 'sashimi bowl', 'calories': 500, 'protein': 35, 'carbs': 60, 'fat': 15},
    77: {'name': 'sushi bowl', 'calories': 550, 'protein': 25, 'carbs': 75, 'fat': 18},
    78: {'name': 'fish-shaped pancake with bean jam', 'calories': 120, 'protein': 3, 'carbs': 25, 'fat': 2},
    79: {'name': 'shrimp with chill source', 'calories': 200, 'protein': 18, 'carbs': 10, 'fat': 10},
    80: {'name': 'roast chicken', 'calories': 300, 'protein': 35, 'carbs': 0, 'fat': 18},

    81: {'name': 'steamed meat dumpling', 'calories': 80, 'protein': 5, 'carbs': 10, 'fat': 3},  # per dumpling
    82: {'name': 'omelet with fried rice', 'calories': 600, 'protein': 25, 'carbs': 70, 'fat': 25},
    83: {'name': 'cutlet curry', 'calories': 700, 'protein': 30, 'carbs': 80, 'fat': 30},
    84: {'name': 'spaghetti meat sauce', 'calories': 400, 'protein': 18, 'carbs': 50, 'fat': 15},
    85: {'name': 'fried shrimp', 'calories': 100, 'protein': 8, 'carbs': 5, 'fat': 6},  # per shrimp
    86: {'name': 'potato salad', 'calories': 180, 'protein': 4, 'carbs': 20, 'fat': 10},
    87: {'name': 'green salad', 'calories': 50, 'protein': 2, 'carbs': 8, 'fat': 1},
    88: {'name': 'macaroni salad', 'calories': 200, 'protein': 5, 'carbs': 25, 'fat': 10},
    89: {'name': 'japanese tofu and vegetable chowder', 'calories': 150, 'protein': 8, 'carbs': 15, 'fat': 7},
    90: {'name': 'pork miso soup', 'calories': 120, 'protein': 10, 'carbs': 8, 'fat': 6},

    91: {'name': 'chinese soup', 'calories': 100, 'protein': 6, 'carbs': 10, 'fat': 4},
    92: {'name': 'beef bowl', 'calories': 650, 'protein': 30, 'carbs': 80, 'fat': 25},
    93: {'name': 'kinpira-style sauteed burdock', 'calories': 120, 'protein': 3, 'carbs': 20, 'fat': 5},
    94: {'name': 'rice ball', 'calories': 150, 'protein': 3, 'carbs': 33, 'fat': 0.5},
    95: {'name': 'pizza toast', 'calories': 300, 'protein': 12, 'carbs': 35, 'fat': 15},
    96: {'name': 'dipping noodles', 'calories': 350, 'protein': 12, 'carbs': 60, 'fat': 8},
    97: {'name': 'hot dog', 'calories': 250, 'protein': 10, 'carbs': 18, 'fat': 15},
    98: {'name': 'french fries', 'calories': 312, 'protein': 4, 'carbs': 41, 'fat': 15},
    99: {'name': 'mixed rice', 'calories': 400, 'protein': 8, 'carbs': 75, 'fat': 10},
    100: {'name': 'goya chanpuru', 'calories': 180, 'protein': 8, 'carbs': 15, 'fat': 12}
}



In [9]:
def predict_nutrition(image_path):
    # Load and preprocess image
    img = Image.open(image_path).convert('RGB')
    img = img.resize(IMG_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) / 255.0

    # Get model prediction
    predictions = model.predict(img_array)
    predicted_class_idx = np.argmax(predictions[0])
    predicted_class_id = predicted_class_idx + 1  # Convert to 1-100 ID system

    # Get nutrition data
    nutrition_data = MANUAL_NUTRITION_DB.get(predicted_class_id,
                                          MANUAL_NUTRITION_DB[100])  # Fallback to last item

    # Adjust for typical serving sizes
    serving_adjustments = {
        'sushi': 4,         # 4 pieces
        'takoyaki': 4,      # 4 pieces
        'tempura': 4,       # 4 pieces
        'croquette': 2,     # 2 pieces
        'pizza': 0.25       # 1 slice
    }

    food_name = nutrition_data['name']
    if food_name in serving_adjustments:
        multiplier = serving_adjustments[food_name]
        adjusted_data = {k: v*multiplier if k != 'name' else v
                        for k, v in nutrition_data.items()}
        nutrition_data = adjusted_data

    return {
        'food_id': predicted_class_id,
        'food_class': nutrition_data['name'],
        'confidence': float(np.max(predictions)),
        'nutrition': {
            'calories': nutrition_data['calories'],
            'protein': nutrition_data['protein'],
            'carbohydrates': nutrition_data['carbs'],
            'fat': nutrition_data['fat']
        }
    }

In [11]:
# Test with sample image
test_image_path = "/content/Screenshot 2025-02-15 173208.png"
result = predict_nutrition(test_image_path)

print("Predicted Class:", result['food_class'])
print("Confidence:", result['confidence'])
print("Nutrition Info:", result['nutrition'])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicted Class: toast
Confidence: 0.9997493624687195
Nutrition Info: {'calories': 75, 'protein': 3, 'carbohydrates': 12, 'fat': 1}


In [13]:
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['ALLOWED_EXTENSIONS'] = {'png', 'jpg', 'jpeg'}

# Load model
model = tf.keras.models.load_model('food_recognition.h5')

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

@app.route('/predict', methods=['POST'])
def handle_prediction():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    file = request.files['file']

    if file.filename == '':
        return jsonify({'error': 'Empty filename'}), 400

    if not allowed_file(file.filename):
        return jsonify({'error': 'Invalid file type'}), 400

    try:
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        result = predict_nutrition(filepath)
        os.remove(filepath)

        return jsonify(result)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
    app.run(host='0.0.0.0', port=5000, threaded=False)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [14]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
def predict_nutrition(image_path):
    img = Image.open(image_path).convert('RGB')
    img = img.resize(IMG_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) / 255.0

    predictions = model.predict(img_array)
    top_class = class_names[np.argmax(predictions[0])]

    if top_class.lower() in nutrition_mapping:
        nutrition = nutrition_mapping[top_class.lower()].to_dict()
    else:
        nutrition = None

    return {
        'food_class': top_class,
        'confidence': float(np.max(predictions)),
        'nutrition': nutrition
    }

In [ ]:
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads/'
model = tf.keras.models.load_model('/content/food_recognition.h5')

@app.route('/predict', methods=['POST'])
def api_predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['file']
    filename = secure_filename(file.filename)
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    file.save(filepath)

    try:
        result = predict_nutrition(filepath)
        return jsonify(result)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)